In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet(path='../geo_data/tz_db/tz_dictionary.parquet')

In [3]:
ext = pd.read_csv(filepath_or_buffer='../geo_data/city_db/worldcities.csv', usecols=['city_ascii', 'population', 'country'])

# names belong to more populated countries take precedence.


exclude_exp = (r'[Rr]epublic|[Ss]tate|[Aa]uthority|Pradesh|'
               r'[Ww]est|[Ee]ast|[Ss]outh|[Nn]orth|[Rr]egion|'
               r'Assam|[Dd]ivision|[Nn]ation|[Qq]uarter|[Pp]rovince|'
               r'[Cc]ontinent|[Cc]ountry|[Dd]istrict|[Ii]sland|[Tt]erritory|[Cc]entral')


In [9]:
ext = pd.read_csv(filepath_or_buffer='../geo_data/city_db/worldcities.csv', 
                  usecols=['city_ascii', 'population'])

merged = df.merge(right=ext, left_on='name', right_on='city_ascii').drop(columns=['name', 'population_x']) \
.rename(columns=dict(city_ascii='city', population_y='population')) \
.astype(dict(city='string', population='Int64', gmt_offset='Int64'))

country_ttl = merged.drop_duplicates(subset=['city', 'country']).groupby('country', as_index=False)['population'].sum()
res = merged.merge(right=country_ttl, on='country') \
.sort_values(by=['population_y', 'population_x'], 
            ascending=False, ignore_index=True)
res['gmt_offset_minute'] = round(res.gmt_offset / 60).astype('Int64')

In [11]:
res.to_parquet('../geo_data/tz_sorted.parquet')